Extracting User's Google Calendar

In [142]:
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
import googleapiclient.discovery
import datetime


In [143]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [144]:
import re
import ollama
import pandas as pd
from ollama import Client
from ollama import chat
import methods
import formats
import json

google_calendar = methods.get_google_calendar_service() #get the google calendar service
tasks = methods.get_task_service() #get the task service
tasks_list = methods.extractTasks(tasks)
calendar = methods.extractCalendar(google_calendar) #extract the calendar from the service
#client = methods.set_up_ChatGPT(calendar, tasks_list) #set up the ChatGPT model
now = methods.current_time()

Logging Previous Conversation

In [145]:
def log_message(role, message):
    filename = f"conversation_log.txt"
    with open(filename, "a") as file:
        timestamp = datetime.datetime.now().isoformat()
        file.write(f"{timestamp} | {role}: {message}\n")

In [146]:
def log_activity(role, message):
    filename = f"activity_log.txt"
    with open(filename, "a") as file:
        timestamp = datetime.datetime.now().isoformat()
        file.write(f"{timestamp} | {role}: {message}\n")

In [147]:
def get_past_conversation():
    filename = f"conversation_log.txt"
    try:
        with open(filename, "r") as file:
            return file.read()
    except FileNotFoundError:
        return ""

Get User's Query

In [148]:
#past_conversation = get_past_conversation()
# schedule tasks or calendar events

context = f"Here's my calendar and To-Do list for the week, store it for your reference. Calendar: {calendar}\n To-Do Tasks: {tasks_list} Today's date and time is: {now}\n"
instruction = """\nAs my trusted virtual assistant, you are an expert in adeptly managing my calendar and assisting with scheduling tasks. Here is what you can do:

1. **Calendar Management**:
 You have the capability to create, adjust, oversee, list, update, delete, and handle recurring events on my calendar as per my requests.
    - In situations where I don't provide enough information, you can ask for more details, upto 4 times, otherwise use your best judgement.
    - Should an event require scheduling or rescheduling and I haven't provided a specific time, consult my calendar and deduce a suitable time slot using your best judgement. Consider my usual habits, such as meal and sleep times, and assume a standard event duration of 1 hour unless your judgement suggests otherwise based on the nature of the event.
    - Before confirming any scheduling actions, scrutinize for any possible scheduling conflicts.
    - Always ask for confirmation before making any changes to the calendar. If I reject the suggested time, offer an alternative time slot.
    - Inform me when you have successfully completed your task and ensure there are no more pending queries from your end, otherwise always end with a question mark.

2. **Task Scheduling**:
You have the capability to schedule tasks from my To-Do list into my calendar.
    - Utilize your best judgement to organize and propose times for scheduling tasks listed in my To-Do list. Prioritize tasks based on urgency and balance throughout the available slots in my schedule.
    - Verify the calendar to ensure a task isn't already scheduled before adding it.
    - Check for any conflicts that may arise due to overlapping tasks or events.
    - Present a thoughtful plan for the scheduled tasks. Use your best judgement to predict task durations based on the nature of each task.
    - Ask for my confirmation before finalizing the schedule.
    - If I disagree with the proposed times, offer alternatives based on my feedback.
    - Always end your messages with a question mark to indicate that you are seeking a response from me, unless it's a confirmation of a task completion.

These are your two functionalities. Based on the user query, perform what is necessary. You are not required to perform both if it is not requested.
Please perform the necessary actions using the Google Calendar API based on the user's query. Only perform what is necessary for the user's query.\n
\n"""
# initial setup

initial_prompt = context + instruction

In [149]:
user_request = input()
combined_prompt = initial_prompt + "\nUser request: " + user_request


In [150]:
def llama2(client, messages):
    response = ollama.chat(model='llama2', messages=messages)
    #print(response)
    return response['message']['content']

messages = [
    {
        "role": "system",
        "content": initial_prompt
    },
    {
        "role": "user",
        "content": user_request
    }
]

log_activity("system", context+instruction)
log_activity("user", user_request)

log_message("user", user_request)

In [151]:
#ollama.pull("llama2")

In [152]:
finalized_response = False
client = Client(host='http://localhost:11434')
#establishing feedback loop to chat with user
while not finalized_response:
    response = llama2(client, messages)
    #print(type(response))
    # response_object = json.loads(response)

    # # Access the content of the message
    # response_content = response_object['message']['content']
    print(response)
    log_message("assistant", response)
    log_activity("assistant", response)
    messages.append({
        "role": "assistant",
        "content": response
    })
    if re.compile(r'\?').search(response):
        finalized_response = False
        user_feedback = input()
        messages.append({
            "role": "user",
            "content": user_feedback
        })
        log_message("user", user_feedback)
        log_activity("user", user_feedback)
    else:
        finalized_response = True


{'model': 'llama2', 'created_at': '2024-04-18T20:28:06.798969Z', 'message': {'role': 'assistant', 'content': 'Of course, I\'d be happy to help! Based on your query, I have scanned your calendar and noticed that you are currently available. I have scheduled a jog for 30 minutes starting at 16:30 today (April 18th). This time slot should give you enough space to complete your jog without any conflicts with other events in your calendar.\n\nWould you like me to bookmark this event so it appears in your calendar? Or would you prefer me to schedule a different time slot for the jog? Please let me know by responding with "bookmark" or "schedule alternative time".'}, 'done': True, 'total_duration': 143391404390, 'load_duration': 10614756281, 'prompt_eval_count': 1046, 'prompt_eval_duration': 103364602000, 'eval_count': 133, 'eval_duration': 29391946000}
Of course, I'd be happy to help! Based on your query, I have scanned your calendar and noticed that you are currently available. I have sched

RequestError: messages must contain content

In [ ]:

instruction2 = """\nBased on the user's request you've just processed, please output the appropriate function call(s) to interact with the Google Calendar API. Format your response as a single string containing one or more function calls separated by semicolons. Do not include explanations or additional text. If there is no action required then output an empty string.
All dates should be in 'YYYY-MM-DD' format, and times should be in 24-hour format 'HH:MM'. Additionally, assume default duration as 1 hour for any event. Use createEvent for scheduling tasks on the to-do list.
Available function templates to include in the response string:
- createEvent(description, date, time, duration)
- moveEvent(event_id, new_date, new_time)
- updateEvent(event_id, new_color, new_duration, new_location)
- deleteEvent(event_id)
- createRecurringEvent(description, start_date, time, duration, frequency, until_date). Frequency can have the value of DAILY, WEEKLY, MONTHLY, YEARLY.
- deleteRecurringEvent(recurring_event_id)
- updateRecurringEvent(recurring_event_id, new_color, new_duration, new_location, new_frequency, new_until)
For all functions other than createEvent and createRecurringEvent, make sure to fill in the id of the event you are referring to from the calendar provided.
Make sure to fill in the required information, such as event names, dates, times, etc., based on the user's request context. Replace placeholders like 'today' or 'tomorrow' with specific dates in the format YYYY-MM-DD. If more details are needed from the user for a function call, use your best judgment to complete the function with the information available.
Example input1: Schedule a team meeting for Friday afternoon at 3pm
Example output1: createEvent("Team Meeting", "YYYY-MM-DD", "15:00", 1:00)
Example input2: Update my team meeting to be two hours
Example output2: updateEvent("event_id", '', 2:00, '')
Example input3: Create a recurring meeting every Wednesday at 2pm until the end of June
Example output3: createRecurringEvent("Meeting", "YYYY-MM-DD", "02:00", "1:00", "WEEKLY", "YYYY-MM-DD")
Example input4: Update my recurring meeting every Wednesday at 2pm to be every month until the end of August
Example output4: updateRecurringEvent("RecurringEventId", "", "02:00", "1:00", "", "MONTHLY", "YYYY-MM-DD")
Example input5: Move my team meeting to tomorrow at 12pm
Example output5: moveEvent("event_id", 'YYYY-MM-DD', '12:00')
Example input6: Clear my afternoon
Example output7: deleteEvent("event_id");deleteEvent("event_id")
Example input8: Schedule all my tasks
Example output8: createEvent("Task 1", "YYYY-MM-DD", "10:00", 1:00);createEvent("Task 2", "YYYY-MM-DD", "11:00", 1:00);createEvent("Task 3", "YYYY-MM-DD", "12:00", 1:00)\n

Now, please generate the necessary function call(s) based on the user's request."""

In [ ]:
messages.append({
    "role": "system",
    "content": instruction2
})
log_activity("system", instruction2)
#print('messages: ', messages)
function_call = llama2(client, messages)
log_activity("assistant", function_call)
print(function_call)
#Example of function call: createEvent("Meeting with John", "2022-12-31", "10:00 AM", 2)

{'model': 'llama2', 'created_at': '2024-04-18T20:16:33.174447Z', 'message': {'role': 'assistant', 'content': 'Based on the user\'s request, the appropriate function call is:\ncreateEvent("Task", "YYYY-MM-DD", "14:00", 1:00)'}, 'done': True, 'total_duration': 105305700116, 'load_duration': 3605500, 'prompt_eval_count': 617, 'prompt_eval_duration': 61738675000, 'eval_count': 43, 'eval_duration': 43508865000}
Based on the user's request, the appropriate function call is:
createEvent("Task", "YYYY-MM-DD", "14:00", 1:00)


In [ ]:
function_calls = [line.strip() for line in function_call.split(';')]
print(function_calls)

# Define a dictionary mapping function names to their corresponding Python functions
functions = {
    "createEvent": methods.createEvent,
    "moveEvent": methods.moveEvent,
    "checkSchedule": methods.checkSchedule,
}

#TODO: works for createEvent, gotta change other functions in methods file to accept arguments as strings and convert them to appropriate data types
# Iterate through the function calls
for call in function_calls:
    # Split the string to extract function name and arguments
    if call == '': continue
    parts = call.split('(')
    function_name = parts[0]

    arguments = parts[1].rstrip(')').split(', ')


    # Call the corresponding function with the provided arguments
    if function_name in functions:
        functions[function_name](*arguments)
    else:
        print(f"Function '{function_name}' not found.")

['Based on the user\'s request, the appropriate function call is:\ncreateEvent("Task", "YYYY-MM-DD", "14:00", 1:00)']


AttributeError: module 'methods' has no attribute 'checkSchedule'